In [0]:
import os
import keras
from keras import *
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
import matplotlib as mpl
from keras import optimizers
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.convolutional import UpSampling2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import img_to_array
from skimage.io import imsave, imread
import matplotlib.pyplot as plt
import numpy as np
import cv2
from scipy import ndimage
from scipy.misc import imresize
from keras.layers.convolutional import Deconv2D as Conv2DTranspose
from keras.models import Model
from keras.layers import Add
from keras.callbacks import ModelCheckpoint,TensorBoard
from keras.optimizers import Adam
from keras.models import Sequential
import sys
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model
from keras.layers import Dropout, Flatten, Dense, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import callbacks
from sklearn.metrics import confusion_matrix
import itertools
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Input
from keras.layers import Input, merge, Dropout, Dense, Lambda, Flatten, Activation
from keras.layers.convolutional import MaxPooling2D, Convolution2D, AveragePooling2D
from keras.layers.normalization import BatchNormalization
from keras.models import Model
from keras.preprocessing import image

from keras.models import Model
from keras import layers
from keras.layers import Dense
from keras.layers import Input
from keras.layers import BatchNormalization
from keras.layers import Activation
from keras.layers import Conv2D
from keras.layers import SeparableConv2D
from keras.layers import MaxPooling2D
from keras.layers import GlobalAveragePooling2D
from keras.layers import GlobalMaxPooling2D
from keras.engine.topology import get_source_inputs
from keras.utils.data_utils import get_file
from keras import backend as K
from keras.applications.imagenet_utils import decode_predictions
from keras.applications.imagenet_utils import _obtain_input_shape

from keras import backend as K
%matplotlib inline

np.random.seed(1337)

In [0]:
resize_h=256
resize_w=256

def takingInput():
    total=len(os.listdir('melanoma/'))+len(os.listdir('others/'))
    print(total)
    imgs=[]
    imgs_mask=[]
    labels=[]
    i = 0
    images = os.listdir('melanoma/')
    for image_name in images:
        image_mask_name = image_name.split('.')[0]
        image_mask_name = image_mask_name+"_segmentation.png"
        img = ndimage.imread('melanoma/'+image_name)       
        #img = cv2.resize(img, (resize_h, resize_w))
        img_mask = ndimage.imread('gt/'+image_mask_name)
        #print(size(imgs_mask))
        #img_mask = cv2.res(img_mask, (array.shape[1], array.shape[0],1))
        #img = np.multiply(img ,img_mask/255.)
        #img = imresize(img,(resize_h,resize_w))
        #ret2,th2 = cv2.threshold(img_mask,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        labels.append(1)
        img = img_to_array(img)
        img_mask=img_to_array(img_mask)
        img_mask=np.reshape(img_mask,(img_mask.shape[0],img_mask.shape[1],1))
        img = np.multiply(img ,img_mask/255.)
        img = imresize(img,(resize_h,resize_w))
        imgs.append(img)
        #imgs_mask.append(img_mask)
        print('Done: {0}/{1} images'.format(i, total))
        i += 1
    images = os.listdir('others/')
    for image_name in images:
        image_mask_name = image_name.split('.')[0]
        image_mask_name = image_mask_name+"_segmentation.png"
        img = ndimage.imread('others/'+image_name)       
        #img = cv2.resize(img, (resize_h, resize_w))
        img_mask = ndimage.imread('gt/'+image_mask_name)
        #print(size(imgs_mask))
        #img_mask = cv2.res(img_mask, (array.shape[1], array.shape[0],1))
        #img = np.multiply(img ,img_mask/255.)
        #img = imresize(img,(resize_h,resize_w))
        #ret2,th2 = cv2.threshold(img_mask,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        labels.append(0)
        img = img_to_array(img)
        img_mask=img_to_array(img_mask)
        img_mask=np.reshape(img_mask,(img_mask.shape[0],img_mask.shape[1],1))
        img = np.multiply(img ,img_mask/255.)
        img = imresize(img,(resize_h,resize_w))
        imgs.append(img)
        print('Done: {0}/{1} images'.format(i, total))
        i += 1
    print('out of for')
    imgs = np.array(imgs, dtype="float")
    imgs_mask = np.array(imgs_mask,dtype="float")
    labels = np.array(labels,dtype='float')
    print('Loading done.')
    return imgs, labels

X, Y = takingInput()

In [0]:
np.save( 'images.npy', X)
np.save('labels.npy', Y)

In [0]:
X = np.load('../imgs_classify.npy')
Y = np.load('../labels.npy')

In [0]:
np.unique(X)

array([  0.00000000e+00,   1.67186529e-16,   2.22915381e-16, ...,
         9.99998152e-01,   9.99998689e-01,   1.00000000e+00])

In [0]:
(trainX, testX, trainY, testY) = train_test_split(X,Y, test_size=0.20, random_state=42)

In [0]:
def f1_score(y_true, y_pred):

    # Count positive samples.
    c1 = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    c2 = K.sum(K.round(K.clip(y_pred, 0, 1)))
    c3 = K.sum(K.round(K.clip(y_true, 0, 1)))

    # If there are no true samples, fix the F1 score at 0.
    if c3 == 0:
        return 0

    # How many selected items are relevant?
    precision = c1 / c2

    # How many relevant items are selected?
    recall = c1 / c3

    # Calculate f1_score
    f1_score = 2 * (precision * recall) / (precision + recall)
    return f1_score

def precision(y_true, y_pred):
    """Precision metric.
    Only computes a batch-wise average of precision.
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [0]:
img_width, img_height = 256, 256
nb_train_samples = 1600
nb_validation_samples = 400
nb_filters1 = 32
nb_filters2 = 64
conv1_size = 3
conv2_size = 2
pool_size = 2
classes_num = 2
batch_size = 64
lr = 0.0002

In [0]:
model = Sequential()
model.add(Conv2D(nb_filters1, (conv1_size, conv1_size), padding="same", input_shape=(img_width, img_height, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size)))

model.add(Conv2D(nb_filters2, (conv2_size, conv2_size), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(pool_size, pool_size), data_format='channels_first'))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='Adam',metrics=['accuracy', f1_score, precision, recall])


In [0]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 256, 256, 32)      896       
_________________________________________________________________
activation_1 (Activation)    (None, 256, 256, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 128, 128, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 128, 128, 64)      8256      
_________________________________________________________________
activation_2 (Activation)    (None, 128, 128, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 128, 64, 32)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 262144)            0         
__________

In [0]:
model_checkpoint1 = ModelCheckpoint('simple_classification1.hdf5',monitor = 'loss', verbose = 1,save_best_only=True)

In [0]:
classifier = model.fit(trainX,trainY, batch_size = 32, epochs = 50, verbose = True, validation_data = (testX, testY),callbacks=[TensorBoard(log_dir='/tmp/gated_cnn_autoencoder', histogram_freq=0,write_graph=True),model_checkpoint1],class_weight={0: 1, 1: 8})

In [0]:
classifier = model.predict(testX)

In [8]:
print("[INFO] accuracy: {:.4f}%, f1_score: {:.4f}, precision: {:.4f}, recall: {:.4f}".format(accuracy*100,f1_score,precision,recall))

[INFO] accuracy: 89.7615%, f1_score: 0.7828, precision: 0.7361, recall: 0.9209
